#**LSTM + Attention**

# I.Import Library





In [1]:

%tensorflow_version 1.x
!pip install keras==2.2.5
!pip install pyvi

import numpy as np
from numpy import random
import os, pickle, re, keras, sklearn, string
from keras.callbacks import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from pyvi import ViTokenizer, ViPosTagger
from keras.layers import *
from keras.models import Model
from keras import optimizers
import gensim, operator, json
import pandas as pd
from sklearn.metrics import *
import keras.backend as K
from keras.models import *
from keras import initializers, regularizers
from keras import optimizers
from keras.engine.topology import Layer
from keras import constraints

TensorFlow 1.x selected.


Using TensorFlow backend.


# II.Read Data

In [2]:
!wget https://thiaisotajppub.s3-ap-northeast-1.amazonaws.com/publicfiles/baomoi.model.bin

--2021-05-09 10:38:09--  https://thiaisotajppub.s3-ap-northeast-1.amazonaws.com/publicfiles/baomoi.model.bin
Resolving thiaisotajppub.s3-ap-northeast-1.amazonaws.com (thiaisotajppub.s3-ap-northeast-1.amazonaws.com)... 52.219.4.63
Connecting to thiaisotajppub.s3-ap-northeast-1.amazonaws.com (thiaisotajppub.s3-ap-northeast-1.amazonaws.com)|52.219.4.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 708212586 (675M) [application/macbinary]
Saving to: ‘baomoi.model.bin.2’

baomoi.model.bin.2  100%[===================>] 675.40M  20.1MB/s    in 36s     

2021-05-09 10:38:46 (18.7 MB/s) - ‘baomoi.model.bin.2’ saved [708212586/708212586]



In [3]:
!wget https://github.com/nthanhkhang/Vietnamese-Social-Media-Emotion-Corpus/raw/main/Data/Data.zip

--2021-05-09 10:38:46--  https://github.com/nthanhkhang/Vietnamese-Social-Media-Emotion-Corpus/raw/main/Data/Data.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nthanhkhang/Vietnamese-Social-Media-Emotion-Corpus/main/Data/Data.zip [following]
--2021-05-09 10:38:46--  https://raw.githubusercontent.com/nthanhkhang/Vietnamese-Social-Media-Emotion-Corpus/main/Data/Data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25932 (25K) [application/zip]
Saving to: ‘Data.zip.1’

Data.zip.1          100%[===================>]  25.32K  --.-KB/s    in 0s      

2021-05-09 10:38:46 (100 MB/s) - ‘Data.zi

In [4]:
import zipfile
with zipfile.ZipFile("Data.zip","r") as zf:
    zf.extractall('Data')
print(zf)

<zipfile.ZipFile [closed]>


In [5]:
path_train ='Data/link1.csv'
path_valid ='Data/link2.csv'
path_test ='Data/link3.csv'
path_stopword = 'Data/stopwords.txt'

# III.Word2vec using baomoi.model.bin

*   Function reading pretrain word embedding library.
*   The word embedding pretrain has been trained in new news, 300-way news



In [6]:
path_embedding= 'baomoi.model.bin'

import io
from gensim.models import KeyedVectors

word_embedding = KeyedVectors.load_word2vec_format(path_embedding, binary=True)
# Example of taking vector of 1 word in the word embedding pretrain
EMBEDDING_DIM = word_embedding['yêu'].shape[0]
print("Embedding: ",EMBEDDING_DIM)
# Vector of love words in pretrained word embedding set.
print(word_embedding['yêu'])


Embedding:  400
[-0.78774583 -0.22327825 -0.6274532  -2.7228408  -2.2186291   0.38002455
  3.8660462   0.9853684  -1.4683082  -1.7013292  -0.5839958  -0.14467287
  3.600142    3.381808   -0.02930526  3.0047843  -0.2006207  -1.0937127
  1.7360235   2.3691583  -0.71597415  3.319453    0.2824182  -3.0814204
  2.6810844  -0.810977    1.5186927  -2.10329     1.3271075  -1.3646411
 -0.11144319 -4.6505136  -1.7251624  -2.31126     1.583203   -0.8746506
 -2.6937015  -1.7733976   0.557898   -1.7562917   1.3282276  -0.3805479
 -1.3979301  -0.1536707  -1.1909302   1.3283668   0.22275637 -2.7959821
 -5.188217   -0.6404673   0.0164395   0.67177856 -1.4948794   0.21867418
 -1.4103376   0.99262404  2.2180524  -0.4881204   3.0988753  -0.31382522
  1.3226501   0.21269594 -1.6409203   1.7758838   2.3379912  -2.4666297
 -0.599687    0.551105   -1.3755493  -1.4293027  -2.6366289   0.40759587
 -0.77850854 -0.6169452  -0.84525913  0.02801617  2.1296268   0.13715844
 -1.1562283  -2.1226277  -0.1346792   0.88

# IV. Data Preprocessing

## 1.Tokenizer

In [7]:
def tokenizer(text):
    token = ViTokenizer.tokenize(text)
    return token

## 2.Delete Icon

In [8]:
def deleteIcon(text):
    text = text.lower()
    s = ''
    pattern = r"[a-zA-ZaăâbcdđeêghiklmnoôơpqrstuưvxyàằầbcdđèềghìklmnòồờpqrstùừvxỳáắấbcdđéếghíklmnóốớpqrstúứvxýảẳẩbcdđẻểghỉklmnỏổởpqrstủửvxỷạặậbcdđẹệghịklmnọộợpqrstụựvxỵãẵẫbcdđẽễghĩklmnõỗỡpqrstũữvxỹAĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXYÀẰẦBCDĐÈỀGHÌKLMNÒỒỜPQRSTÙỪVXỲÁẮẤBCDĐÉẾGHÍKLMNÓỐỚPQRSTÚỨVXÝẠẶẬBCDĐẸỆGHỊKLMNỌỘỢPQRSTỤỰVXỴẢẲẨBCDĐẺỂGHỈKLMNỎỔỞPQRSTỦỬVXỶÃẴẪBCDĐẼỄGHĨKLMNÕỖỠPQRSTŨỮVXỸ,._]"
    for char in text:
        if char !=' ':
            if len(re.findall(pattern, char)) != 0:
                s+=char
            elif char == '_':
                s+=char
        else:
            s+=char
    s = re.sub('\\s+',' ',s)
    return s.strip()

## 3.Clean Doc

In [9]:
def clean_doc(doc):
    doc = tokenizer(doc)
    for punc in string.punctuation:# delete all punctuation (!,? ..) in a sentence
        if punc != "_":
            doc = doc.replace(punc,' ')
    doc = deleteIcon(doc) 
    doc = re.sub(r"[0-9]+", " num ", doc)# Delete numbers
    doc = doc.lower()#lowercase 
    doc = re.sub('\\s+',' ',doc)# Remove lots of spaces
    return doc

## 4.Stopword

In [10]:
# from underthesea import word_tokenize
def pre_process(questions):
    stop_words = stopwords.words("english")
    questions_stop = [[t for t in tokens if (t not in stop_words) and (3 < len(t.strip()) < 15)]
                      for tokens in questions_tokens]
    questions_stop = pd.Series(questions_stop)
    return questions_stop

## 5.Word Segmentation

# V.Train/Test data

In [11]:
train_data = pd.read_csv(path_train,encoding='utf-16')
valid_data = pd.read_csv(path_valid,encoding='utf-16')
test_data = pd.read_csv(path_test,encoding='utf-16')

X_train = train_data["Sentence"].apply(lambda x : clean_doc(x))
y_train = train_data["Emotion"]

X_val = valid_data["Sentence"].apply(lambda x : clean_doc(x))
y_val = valid_data["Emotion"]

X_test = test_data["Sentence"].apply(lambda x : clean_doc(x))
y_test = test_data["Emotion"]

In [12]:
print(len(X_train),len(y_train))
print(len(X_val),len(y_val))
print(len(X_test),len(y_test))


100 100
100 100
100 100


## 1.Catalog vector

In [13]:
classes = ['Anger','Disgust','Enjoyment','Fear','Other','Sadness','Surprise']
def to_category_vector(label):
    vector = np.zeros(len(classes)).astype(np.float64)
    index = classes.index(label)
    vector[index] = 1.0
    return vector

## 2.Convert labels to numbers in train and test practice

In [14]:
y_train_encode = []
for label in y_train:
    y_train_encode.append(to_category_vector(label))


y_val_encode = []
for label in y_val:
    y_val_encode.append(to_category_vector(label))

print(classes)
print(y_train_encode[0])
print(y_train[0])

['Anger', 'Disgust', 'Enjoyment', 'Fear', 'Other', 'Sadness', 'Surprise']
[0. 0. 1. 0. 0. 0. 0.]
Enjoyment


## 3.LSTM


*   All the words in the X_train set will form a dictionary
*   Each vector of the input word, it will turn into a vector with a fixed number of dimensions and each vocabulary will be replaced by its index in the dictionary
* Number of vector dimensions per input we will take the longest sentence which is the direction of the vector and the shorter arcs will automatically add the value 0 after

In [15]:
xLengths = [len(x.split(' ')) for x in X_train]
h = sorted(xLengths)  #sorted lengths
maxLength =h[len(h)-1]
print("The longest sentence length value: ",maxLength)
input_tokenizer = Tokenizer(filters="",oov_token="UNK")
input_tokenizer.fit_on_texts(X_train)
input_vocab_size = len(input_tokenizer.word_index) + 1
word_index = input_tokenizer.word_index
print("input_vocab_size:",input_vocab_size)
X_train_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences(X_train), maxlen=maxLength,padding="post"))

The longest sentence length value:  114
input_vocab_size: 452


## 4.Enter the example using LSTM

In [16]:
print("Input String : ", X_train[0])
print("Encode : ",X_train_encode[0])

X_val_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences(X_val), maxlen=maxLength,padding="post"))

Input String :  một giọng nói của người trải qua sự bi_đát mong được lên tv some day
Encode :  [ 39  18  57  22  40 176  82 112 113  14  23   5  10 114 115   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0]


## 5.Generate Embedding
Function takes the vector of vocabulary in pre-trained word embedding

In [17]:
def generate_embedding(word_index, model_embedding,EMBEDDING_DIM):
    count6 = 0
    countNot6 = 0
    #embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM)) 
    embedding_matrix = np.asarray([np.random.uniform(-0.01,0.01,EMBEDDING_DIM) for _ in range((len(word_index) + 1))])
    list_oov = []
    word_is_trained = []
    for word, i in word_index.items():
        try:
            embedding_vector = model_embedding[word]
            word_is_trained.append(word)
        except:
            continue
        if embedding_vector is not None:
            count6 +=1
            embedding_matrix[i] = embedding_vector
    
    print('Number of words in pre-train embedding: ' + str(count6))
    print('Number of words not in pre-train embedding: ' + str(countNot6))
    return embedding_matrix,word_is_trained

In [18]:
embedding_matrix,word_is_trained = generate_embedding(word_index,word_embedding,EMBEDDING_DIM)
print(word_is_trained)

Number of words in pre-train embedding: 424
Number of words not in pre-train embedding: 0
['anh', 'em', 'cho', 'lên', 'đi', 'ơi', 'là', 'rồi', 'tv', 'a', 'chú', 'có', 'mong', 'tuna', 'không', 'chị', 'giọng', 'tuân', 'e', 'thì', 'của', 'được', 'vợ', 'nhà', 'cháu', 'năm', 'ko', 'còn', 'để', 'với', 'khi', 'ạ', 'show', 'và', 'bị', 'đã', 'con', 'một', 'người', 'gì', 'thấy', 'cừu', 'hay', 'xem', 'nữa', 'mặt', 'tivi', 'lần', 'nghe', 'kể', 'sống', 'video', 'thứ', 'ra', 'nào', 'nói', 'đến', 'ông', 'như', 'giống', 'xong', 'phải', 'đấy', 'cái', 'nhưng', 'lắm', 'chuyện', 'nay', 'cay', 'vào', 'sofa', 'này', 'yêu', 'ngủ', 'ngoài', 'đc', 'vài', 'về', 'thật', 'sau', 'qua', 'nhất', 'có_thể', 'cũng', 'nghĩ', 'nên', 'mình', 'nhau', 'tuổi', 'đó', 'làm', 'mẹ', 'clip', 'someday', 'lâu', 'thôi', 'mà', 'ngày', 'năm_ngoái', 'live', 'nhớ', 'group', 'chờ', 'biết', 'vì', 'nha', 'r', 'k', 'quen', 'hết', 'sự', 'bi_đát', 'some', 'day', 'thích', 'cj', 'phết', 'lời', 'buồn', 'bây_giờ', 'vs', 'vui', 'anh_em', 'v', 'quý

# VI.Attention Layer

In [19]:
def dot_product(x, kernel):
	if K.backend() == 'tensorflow':
		return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
	else:
		return K.dot(x, kernel)

class AttentionWithContext(Layer):
	def __init__(self,
				 W_regularizer=None, u_regularizer=None, b_regularizer=None,
				 W_constraint=None, u_constraint=None, b_constraint=None,
				 bias=True, **kwargs):

		self.supports_masking = True
		self.init = initializers.get('glorot_uniform')

		self.W_regularizer = regularizers.get(W_regularizer)
		self.u_regularizer = regularizers.get(u_regularizer)
		self.b_regularizer = regularizers.get(b_regularizer)

		self.W_constraint = constraints.get(W_constraint)
		self.u_constraint = constraints.get(u_constraint)
		self.b_constraint = constraints.get(b_constraint)

		self.bias = bias
		super(AttentionWithContext, self).__init__(**kwargs)

	def build(self, input_shape):
		assert len(input_shape) == 3

		self.W = self.add_weight((input_shape[-1], input_shape[-1],),
								 initializer=self.init,
								 name='{}_W'.format(self.name),
								 regularizer=self.W_regularizer,
								 constraint=self.W_constraint)
		if self.bias:
			self.b = self.add_weight((input_shape[-1],),
									 initializer='zero',
									 name='{}_b'.format(self.name),
									 regularizer=self.b_regularizer,
									 constraint=self.b_constraint)

		self.u = self.add_weight((input_shape[-1],),
								 initializer=self.init,
								 name='{}_u'.format(self.name),
								 regularizer=self.u_regularizer,
								 constraint=self.u_constraint)

		super(AttentionWithContext, self).build(input_shape)

	def compute_mask(self, input, input_mask=None):
		# do not pass the mask to the next layers
		return None

	def call(self, x, mask=None):
		uit = dot_product(x, self.W)

		if self.bias:
			uit += self.b

		uit = K.tanh(uit)
		ait = dot_product(uit, self.u)

		a = K.exp(ait)

		if mask is not None:
			a *= K.cast(mask, K.floatx())
		a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

		a = K.expand_dims(a)
		weighted_input = x * a
		
		return weighted_input

	def compute_output_shape(self, input_shape):
		return input_shape[0], input_shape[1], input_shape[2]
	
class Addition(Layer):
	def __init__(self, **kwargs):
		super(Addition, self).__init__(**kwargs)

	def build(self, input_shape):
		self.output_dim = input_shape[-1]
		super(Addition, self).build(input_shape)

	def call(self, x):
		return K.sum(x, axis=1)

	def compute_output_shape(self, input_shape):
		return (input_shape[0], self.output_dim)

## 1.Build mode LSTM

In [20]:
filter_nums = 256 # best 128
def build_model():
        inputs  = Input(shape=(maxLength, ), dtype='float64', name='inputs')    
        embedding_layer = Embedding(input_vocab_size,EMBEDDING_DIM,weights=[embedding_matrix], input_length=maxLength, trainable=True,name = 'word_emb')(inputs)
        embedding_layer = SpatialDropout1D(0.75)(embedding_layer)
                
              
        lstm_feature1 = CuDNNLSTM(filter_nums, return_sequences=True)(embedding_layer)

        att1 = AttentionWithContext()(lstm_feature1)
        att1 = Addition()(att1)

        fc1 = Dropout(0.5)(Dense(256, name = 'dense_1')(att1))
        output1 = Dense(len(classes),name="output1", activation='softmax')(fc1)

    
        # define optimizer

        model = Model(inputs=inputs, outputs=output1)
        tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
        
        model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        
        history = model.fit(X_train_encode, np.array(y_train_encode), validation_data = (X_val_encode,np.array(y_val_encode)) , batch_size=50, epochs=100,callbacks=[tensorBoardCallback])
        return model

model = build_model()











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 100 samples, validate on 100 samples


Epoch 1/100
100/100 [==============================] - 7s 69ms/step - loss: 1.8981 - acc: 0.3600 - val_loss: 1.9148 - val_acc: 0.1900

Epoch 2/100
100/100 [==============================] - 0s 807us/step - loss: 1.7388 - acc: 0.5000 - val_loss: 2.2418 - val_acc: 0.1900
Epoch 3/100
100/100 [==============================] - 0s 684us/step - loss: 1.5215 - acc: 0.5300 - val_loss: 3.1445 - val_acc: 0.1900
Epoch 4/100
100/100 [==============================] - 0s 670us/step - loss: 1.4201 - acc: 0.5300 - val_loss: 3.3296 - val_acc: 0.1900
Epoch 5/100
100/100 [==============================] - 0s 648us/step - loss: 1.4435 - acc: 0.5300 - val_loss: 2.7502 - val_acc: 0.1900
Epoch 6/100
100/100 [==============================] -

## 2.Predict the results

In [21]:
X_test_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences(X_test), maxlen=maxLength,padding="post"))
test_length = len(X_test_encode)

y_predict = []
predicted = model.predict(X_test_encode)
for predict in predicted:
    index2, value = max(enumerate(predict), key=operator.itemgetter(1))
    y_predict.append(classes[index2])
    
print(y_predict[0])

Sadness


In [22]:
print(y_predict)

['Sadness', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Other', 'Other', 'Other', 'Other', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Other', 'Sadness', 'Other', 'Other', 'Enjoyment', 'Sadness', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Sadness', 'Other', 'Enjoyment', 'Enjoyment', 'Other', 'Enjoyment', 'Enjoyment', 'Sadness', 'Enjoyment', 'Other', 'Sadness', 'Enjoyment', 'Enjoyment', 'Sadness', 'Enjoyment', 'Other', 'Sadness', 'Enjoyment', 'Other', 'Other', 'Enjoyment', 'Enjoyment', 'Sadness', 'Enjoyment', 'Sadness', 'Other', 'Other', 'Disgust', 'Enjoyment', 'Other', 'Other', 'Other', 'Enjoyment', 'Other', 'Enjoyment', 'Disgust', 'Other', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Disgust', 'Enjoyment', 'Other', 'Other', 'Enjoyment', 'Enjoyment', 'Other', 'Enjoyment', 'Enjoyment', 'Other', 'Other', 'Other', 'Disgust', 'Enjoyment', 'Enjoyment', 'Other', 'Enjoyment', 'Surprise', 'Enjoyment', 'Other', 'Disgust', 'Enjoyment', 'Other', 'Other', 'Sadness', 'Enjoyment', 'Sadness

## 3.Report the performance metrics (Accuracy, F1-score...)

In [23]:
precision = precision_score(y_test, y_predict, average='weighted')
recall = recall_score(y_test, y_predict, average='weighted')
f1score = f1_score(y_test, y_predict, average='micro')
accuracy = accuracy_score(y_test, y_predict)

print("Result model LSTM + Attention layer")
print("Results of the models")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-Score: ", f1score)
print("Accuracy: ", accuracy)

print(classification_report(y_test,y_predict))

Result model LSTM + Attention layer
Results of the models
Precision:  0.1440322580645161
Recall:  0.18
F1-Score:  0.18
Accuracy:  0.18
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00         7
     Disgust       0.14      0.05      0.07        21
   Enjoyment       0.25      0.35      0.29        34
        Fear       0.00      0.00      0.00         1
       Other       0.16      0.28      0.20        18
     Sadness       0.00      0.00      0.00         3
    Surprise       0.00      0.00      0.00        16

    accuracy                           0.18       100
   macro avg       0.08      0.10      0.08       100
weighted avg       0.14      0.18      0.15       100



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# VII.Enter the demo program into 1 sentence

In [24]:
def demo(str):
  demo_pre = clean_doc(str)
  X_demo_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences([demo_pre]), maxlen=maxLength,padding="post"))
  predicted = model.predict(X_demo_encode)
  index2, value = max(enumerate(predicted[0]), key=operator.itemgetter(1))
  print(str)
  print("Predict the results:", classes[index2])

In [25]:
demo('một giọng nói của người trải qua sự bi đát.mong được lên tv some day')

một giọng nói của người trải qua sự bi đát.mong được lên tv some day
Predict the results: Enjoyment
